Nous allons détailler les fonctions dans ce notebook.

**IMPORT**

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import Path

In [2]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"
url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

In [3]:
driver_path = str(Path("../chromedriver.exe").resolve())

In [4]:
driver_path

'/home/thomas/code/thomasgassin/kindle-scrapper/chromedriver.exe'

In [5]:
browser = webdriver.Chrome(executable_path =  driver_path)
browser.get(url)

/tmp/ipykernel_18434/47265324.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path =  driver_path)


WebDriverException: Message: Can not connect to the Service /home/thomas/code/thomasgassin/kindle-scrapper/chromedriver.exe


# Trouver l'url des livres sur une page données

In [11]:
response = requests.get(url_kindle_cat_art_and_photo_subcat_umusic_p2)
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
soup

<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Sorry! Something went wrong!</title>
<style>
  html, body {
    padding: 0;
    margin: 0
  }

  img {
    border: 0
  }

  #a {
    background: #232f3e;
    padding: 11px 11px 11px 192px
  }

  #b {
    position: absolute;
    left: 22px;
    top: 12px
  }

  #c {
    position: relative;
    max-width: 800px;
    padding: 0 40px 0 0
  }

  #e, #f {
    height: 35px;
    border: 0;
    font-size: 1em
  

In [ ]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

# Sur une page récupérer les données : de classement de vente , nom du livre catérgorie